In [13]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [25]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
# We zip the latitudes and longitudes into tuples using the zip() function. Returns a ZIP object
lat_lngs = zip(lats, lngs)

In [26]:
# Add the latitudes and longitudes to a list.
# Here, we are casting the lat_lngs Zip object to a List, so that we can index it appropriately later
coordinates = list(lat_lngs)
print(coordinates)

683974, 22.95838278497405), (63.57662379388097, 52.447525042232), (72.80866963535689, -113.30058468794418), (84.07756904151768, -117.50979592570326), (-52.6800081148603, 50.07885180501762), (-57.40236465187255, 161.9569166181471), (58.613183380589646, 63.39000926929654), (-12.05356668972118, -99.22674407835078), (-26.691955312628764, -30.392333318456252), (3.1920717397889575, 143.4071385281777), (-13.88847645233038, 45.69240227322999), (86.656982015327, 109.67567381914807), (77.66247713344666, 104.50038866041803), (-15.469208939882094, 48.11195198131199), (-78.4463094497693, 13.908936864505534), (-16.66867283319776, 162.82938863925114), (-13.92730093244991, 121.25786329481258), (-11.927972599641677, 141.70416599742742), (-77.57906159046006, -75.85594426785337), (-28.390391209627367, -25.905639455409215), (-86.74454190072291, -149.0925481985075), (1.2866360358552384, -106.22500117856734), (5.618934137070909, -137.54284172186345), (-5.058363675500061, 120.6935358750763), (15.247750601309

In [5]:
# Creating a random list of cities using citypy -----------------------------------------

# Open source, docs on Github: https://github.com/wingchen/citipy
# Citipy allows us to pass in coordinates and returns a city name, as a city object

from citipy import citipy

In [6]:
# Practice getting a city and country code by passing in randomly generated latitudes and longitudes

# Loop through 5/1500 values in the coordinates list
for coordinate in coordinates[:5]:
    # using the nearest_city function from the citipy module, we pass in the first and second coordinates (latitude, longitude) and print the city_name attribute
    print(citipy.nearest_city(coordinate[0],coordinate[1]).city_name,
    # we do the same, but return country_code attribute
    citipy.nearest_city(coordinate[0],coordinate[1]).country_code)

pullman us
noumea nc
faanui pf
dikson ru
hermanus za


In [27]:
# We can use the above methodology to pass in our randomly generated coordinates and return city names, which we can use the OpenWeather API to access weather of 

# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination. Loop through each coordinates in our coordinates list above
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list. From our list of randomly generated coordinates, we might have duplicated cities!
    # Important to account for edge cases
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

607